# Concurrency

Python can speed up the execution of code in some instances by leveraging concurrency which is the ability for a program to manage multiple tasks at the same time. It doesn’t necessarily mean that tasks are running simultaneously, but rather that the program can make progress on several tasks by switching between them efficiently. This is especially useful when some tasks are waiting — for example, on input, a network response, or a file operation. Concurrency allows a system to remain responsive and efficient, even when handling many operations.

### Multiprocessing
Multiprocessing is where multiple tasks are split across different computer cores (either CPU or GPU) so that they may be executed simultaneously. This can greatly speed up the execution of code since the program can progress multiple tasks at the same time rather than waiting for each to finish before starting the next. It is sometimes refered to as parallelisation since multiple tasks are being executed in parallel. 

One use-case where multiprocessing is useful is when iterating over a list and applying a computationally expensive (i.e. slow) function to each element in the list. In the example below, we first loop over a range and apply the computionally expensive function without multiprocessing. So each execution will only begin after the previous execution has finished. Next we apply multiprocessing to allow multiple executions to happen simulataneously. In both instance we time the execution and compare the improvement. Note that you will need to be running this code on a machine with multiple CPU cores in order to see an improvement in execution time. 

In [9]:
import time
from multiprocessing import Pool, cpu_count

numbers = range(5)
num_cores = cpu_count()
print("Number of CPU cores available:", num_cores)

def computationally_expensive_function(a):
    time.sleep(3)
    return a

sequential__start_time = time.time()
sequential__results = []
for i in numbers:
    a = computationally_expensive_function(i)
    sequential__results.append(a)
sequential__end_time = time.time()

parallel_start_time = time.time()
# Create a multiprocessing pool with processes set to the number of CPUs
with Pool(processes=num_cores) as p:
    parallel_results = p.map(computationally_expensive_function, numbers)
parallel_end_time = time.time()

print("Non-parallel execution time:", round(sequential__end_time - sequential__start_time, 2), "seconds")
print("Parallel execution time:", round(parallel_end_time - parallel_start_time, 2), "seconds")

Number of CPU cores available: 4
Non-parallel execution time: 15.01 seconds
Parallel execution time: 6.03 seconds


### Multithreading
Multithreading is where multiple tasks (called threads) are executed within the same process and share the same memory space. Threads are lightweight and can be used to perform multiple operations concurrently, particularly when some operations are slow due to waiting for external resources (such as reading from disk, downloading files, or waiting on user input). Unlike multiprocessing, multithreading does not typically provide a performance boost for CPU-heavy tasks in Python due to the Global Interpreter Lock (GIL), which allows only one thread to execute Python bytecode at a time.

Multithreading is particularly useful for I/O-bound tasks — that is, tasks where the program spends much of its time waiting, rather than using the CPU. In the example below, we first simulate a blocking I/O operation (like downloading files) using a loop, where each task runs one after another. Then, we use multithreading to run several of these tasks concurrently, allowing the program to perform other work while waiting on the blocking operations. This results in faster overall execution for I/O-bound workloads.

While multiprocessing spreads tasks across multiple CPU cores for true parallelism, multithreading uses concurrency within a single core to make better use of idle time — making it ideal when performance is limited by delays, not computation.

In [11]:
import time
import threading

# Simulated I/O-bound task (e.g., downloading a file)
def io_bound_task(n):
    print(f"Starting task {n}")
    time.sleep(2)  # Simulate waiting (e.g., for I/O)
    print(f"Finished task {n}")


sequential_start_time = time.time()

for i in range(5):
    io_bound_task(i)

sequential_end_time = time.time()

print(f"\nSequential execution took {sequential_end_time - sequential_start_time:.2f} seconds\n")

multi_threaded_start_time = time.time()

threads = []
for i in range(5):
    thread = threading.Thread(target=io_bound_task, args=(i,))
    thread.start()
    threads.append(thread)

# Wait for all threads to complete
for thread in threads:
    thread.join()

multi_threaded_end_time = time.time()

print(f"\nMultithreaded execution took {multi_threaded_end_time - multi_threaded_start_time:.2f} seconds")


Starting task 0
Finished task 0
Starting task 1
Finished task 1
Starting task 2
Finished task 2
Starting task 3
Finished task 3
Starting task 4
Finished task 4

Sequential execution took 10.01 seconds

Starting task 0
Starting task 1
Starting task 2
Starting task 3
Starting task 4
Finished task 0Finished task 1

Finished task 4
Finished task 2
Finished task 3

Multithreaded execution took 2.00 seconds
